In [34]:
# read data/images-tem.json
import pandas as pd
import json

# df display options
pd.set_option('max_colwidth',100)

# read json file
with open('data/images-tem.json') as f:
    data = json.load(f)

# read json file into pandas dataframe
df = pd.json_normalize(data, meta=['metaData'])

# clean up dataframe
df.rename(columns={'metaData.doi': 'doi'}, inplace=True)
df.rename(columns={'metaData.title': 'title'}, inplace=True)
df.rename(columns={'metaData.id': 'id'}, inplace=True)
df.rename(columns={'microscopyType': 'microscopy_type'}, inplace=True)
df.drop(columns=['metaData.keywords'], inplace=True)


In [35]:
# print dataframe columns in a list
print(f'Columns: {list(df.columns)}')

Columns: ['file', 'description', 'microscopy_type', 'type', 'title', 'id', 'doi']


In [33]:
# number of unique DOIs
print(f'Number of unique DOIs: {len(df["doi"].unique())}')

Number of unique DOIs: 63


In [ ]:

# group by id show only the ones with more than 1, show only the file column and id column show first 10 rows
# show count of unique
# df.groupby('id').filter(lambda x: len(x) > 1)[['file', 'id']].value_counts()
# df.groupby('doi').head(15)

# show entire file name
# df.groupby('id').filter(lambda x: len(x) > 1)[['file']]
# df.groupby('id').filter(lambda x: len(x) > 1)